In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from transformer_lens import utils
from sprint.loading import load_all

/opt/conda/envs/sprint/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model, data, sae = load_all()

Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([215402, 128]), dtype: torch.int64, device: cuda:0
{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 2048,
 'dict_mult': 8,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'lr': 0.0001,
 'model_batch_size': 512,
 'num_tokens': 2000000000,
 'seed': 52,
 'seq_len': 128}
Encoder device: cuda:0


In [8]:
def ln2_mlp_until_post(x, ln, mlp):
    x = ln(x)
    x = x @ mlp.W_in + mlp.b_in
    x = mlp.act_fn(x)
    return x

In [23]:
out, cache = model.run_with_cache(data[:32])
mid_acts = cache[utils.get_act_name("resid_mid", 0)]
print(mid_acts.shape)

torch.Size([32, 128, 512])


In [27]:
x = torch.rand(1, 1, 512).cuda()

ln2_mlp_until_post(x, model.blocks[0].ln2, model.blocks[0].mlp).shape

torch.Size([1, 1, 2048])

In [26]:
model.blocks[0].mlp(model.blocks[0].ln2(x)).shape

torch.Size([1, 1, 512])

In [38]:
model.blocks[0].mlp._parameters.keys()

odict_keys(['W_in', 'b_in', 'W_out', 'b_out'])

In [39]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resi

In [126]:
from torch import gradient
from transformer_lens.utils import get_act_name

# Forward
start_output = None
def start_fwd_hook(value, hook):
    global start_output
    start_output = value.clone().detach()
    start_output.requires_grad_(True)
    print("Forward hook triggered")
    return value

end_output = None
def end_fwd_hook(value, hook):
    global end_output
    end_output = value.clone().detach()
    return value


# # Backward
# gradient_at_start = None


# def end_bwd_hook(value, hook):
#     global gradient_at_start
#     print("Backward hook triggered")
#     if start_output.grad is not None:
#         gradient_at_start = start_output.grad.clone()
#         print("Gradient captured")
#     else:
#         print("Gradient not captured")


x = model.run_with_hooks(
    data[0:1, 0],
    # fwd_hooks=[("blocks.0.ln2.hook_normalized", start_fwd_hook)],
    # bwd_hooks=[("blocks.0.mlp.hook_post", end_bwd_hook)],
    fwd_hooks=[
        ("blocks.0.mlp.hook_post", start_fwd_hook),
        ("blocks.0.ln2.hook_normalized", end_fwd_hook),
    ],
    # bwd_hooks=[("blocks.0.ln2.hook_normalized", end_bwd_hook)],
    reset_hooks_end=False,
    # return_type="loss",
)
(end_output @ start_output).backward(gradient=start_output)
# print(gradient_at_start)

Forward hook triggered
Forward hook triggered
Forward hook triggered
Forward hook triggered
Forward hook triggered


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [1, 512] but got: [1, 1].

In [119]:
print(end_output.shape)
print(start_output.shape)

torch.Size([32, 128, 512])
torch.Size([32, 128, 2048])


In [121]:
_, cache = model.run_with_cache(
    data[:32, 0], 
    # names_filter=["blocks.0.mlp.hook_post", "blocks.0.ln2.hook_normalized"], 
    names_filter = ["blocks.0.ln2.hook_normalized", "blocks.0.mlp.hook_post"],
    incl_bwd=True,
    return_type="loss"
)

Forward hook triggered
Forward hook triggered
Forward hook triggered
Forward hook triggered
Forward hook triggered
Forward hook triggered
Forward hook triggered
